In [1]:
from openai import OpenAI
import numpy as np

In [12]:
client = OpenAI(api_key="") #Removed my personal key from here

In [13]:
with open('messi.txt', 'r') as f:
    text = f.read()
    print(text)

Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987), also known as Leo Messi, is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players of all time, Messi set numerous records for individual accolades won throughout his professional footballing career such as eight Ballon d'Or awards and eight times being named the world's best player by FIFA.[note 2] He is the most decorated player in the history of professional football having won 45 team trophies,[note 3] including twelve league titles, four UEFA Champions Leagues, two Copa Américas, and one FIFA World Cup. Messi holds the records for most European Golden Shoes (6), most goals for a single club (672, with Barcelona), most goals (474), hat-tricks (36) and assists (192) in La Liga, most matches played (39), assists (18) and goal contributions (34) 

In [14]:
CHUNNK_SIZE = 100
chunks = []
current_place = 0
while current_place < len(text):
    current_chunk = text[current_place : current_place + CHUNNK_SIZE]
    chunks.append(current_chunk)
    current_place += CHUNNK_SIZE
print(chunks)

['Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987), a', 'lso known as Leo Messi, is an Argentine professional footballer who plays as a forward for and capta', 'ins both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as on', 'e of the greatest players of all time, Messi set numerous records for individual accolades won throu', "ghout his professional footballing career such as eight Ballon d'Or awards and eight times being nam", "ed the world's best player by FIFA.[note 2] He is the most decorated player in the history of profes", 'sional football having won 45 team trophies,[note 3] including twelve league titles, four UEFA Champ', 'ions Leagues, two Copa Américas, and one FIFA World Cup. Messi holds the records for most European G', 'olden Shoes (6), most goals for a single club (672, with Barcelona), most goals (474), hat-tricks (3', '6) and assists (192) in La Liga, most matches played (39), ass

In [22]:
embedded_chunks = []
for chunk in chunks:
    response = client.embeddings.create(
        input=chunk,
        model="text-embedding-3-small"
    )
    embedded_chunks.append( (chunk, response.data[0].embedding) )

In [43]:
question = input(" ") # Have question input instead of the hard coded ones
question_embedding = client.embeddings.create(
    input=question,
    model="text-embedding-3-small"
).data[0].embedding

 When was Messi born


In [44]:
def cosine_similarity(A, B):
    return np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))

In [45]:
best_cosine_similarity = 0
index = 0
for chunk in embedded_chunks:
    chunk_text, chunk_embedding = chunk
    similarity = cosine_similarity(question_embedding, chunk_embedding)
    if similarity > best_cosine_similarity:
        best_cosine_similarity = similarity
        bestChunk = index
    index += 1

In [46]:
print(f"Relevant information: {embedded_chunks[bestChunk][0]}")

completion = client.chat.completions.create(
    model="gpt-3.5-turbo", # Used 3.5 so I don't get taxed but this can be changed for testing
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user", "content": f"User question: {question}. Potentially useful information: {embedded_chunks[bestChunk][0]}"
        }
    ]
)

Relevant information: Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987), a


In [47]:
print(completion.choices[0].message.content)

Messi was born on June 24, 1987.
